# Test part of data pipeline where we merge all the hourly data

Breaks on full run: need to convert to dask df

In [ ]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
from src.load_data import PATH_TO_LOCAL_REPO

In [ ]:
# Start client so can see worker mem use
from dask.distributed import Client 
client = Client(n_workers=2, threads_per_worker=2, memory_limit='1GB') # limit worker size to create issues even on --small dataset
client

In [ ]:
%reload_ext autoreload
%autoreload 2

# # Tell python where to look for modules.
import sys

sys.path.append('../../hourly-egrid/')

# import local modules
import src.data_cleaning as data_cleaning

In [ ]:
o_shaped_eia_data = pd.read_csv(PATH_TO_LOCAL_REPO + 'data/outputs/small/shaped_eia923_data_2020.csv', parse_dates=['datetime_utc'])
o_partial_cems_scaled = pd.read_csv(PATH_TO_LOCAL_REPO + 'data/outputs/small/partial_cems_scaled_2020.csv', parse_dates=['datetime_utc']) #NOT FINAL VERSION 
o_cems = pd.read_csv(PATH_TO_LOCAL_REPO + 'data/outputs/small/cems_2020.csv', parse_dates=['datetime_utc']) # NOT FINAL VERSION

# shaped_eia_data = pd.read_csv(PATH_TO_LOCAL_REPO + 'data/outputs/shaped_eia923_data_2020.csv')
# partial_cems_scaled = pd.read_csv(PATH_TO_LOCAL_REPO + 'data/outputs/partial_cems_scaled_2020.csv') #NOT FINAL VERSION 
# cems = pd.read_csv(PATH_TO_LOCAL_REPO + 'data/outputs/cems_2020.csv') # NOT FINAL VERSION

In [ ]:
# most of the stuff done to partial_cems_scaled and cems in data_pipeline is in plant_static_attributes: 
#plant_static_attributes = pd.read_csv(PATH_TO_LOCAL_REPO + 'data/results/plant_data/plant_static_attributes.csv')

plant_static_attributes = pd.read_csv(PATH_TO_LOCAL_REPO + 'data/results/small/plant_data/plant_static_attributes.csv')

In [ ]:
o_partial_cems_scaled = o_partial_cems_scaled.merge(plant_static_attributes, how='left', on='plant_id_eia')
o_cems = o_cems.merge(plant_static_attributes, how='left', on='plant_id_eia')

In [ ]:
# Problem data_pipeline steps: on small data, these are fine with worker size 16GB (break with worker size 1GB)
# On full dataset, these break with worker size 16GB (ie Gailin's laptop)

combined_plant_data = data_cleaning.combine_subplant_data(
        o_cems, o_partial_cems_scaled, o_shaped_eia_data
)

# 12. Aggregate CEMS data to BA-fuel and write power sector results
ba_fuel_data = data_cleaning.aggregate_plant_data_to_ba_fuel(
    combined_plant_data, plant_static_attributes
)

In [ ]:

#plants = combined_plant_data.groupby('plant_id_eia').sum().compute()

# What is the minimum dataset for which this breaks? 

In [ ]:
o_shaped_eia_data.datetime_utc = pd.to_datetime(o_shaped_eia_data.datetime_utc)

In [ ]:
#dt_shaped['datetime_utc'] = dt_shaped.datetime_utc.dt.to_datetime64()


In [ ]:
dt_shaped = o_shaped_eia_data.set_index('datetime_utc')
shaped_eia_dask = dd.from_pandas(dt_shaped, npartitions=50)

In [ ]:
shaped_eia_dask

In [ ]:
smol = shaped_eia_dask[['net_generation_mwh','fuel_consumed_mmbtu']]

In [ ]:
# even this breaks. If can get this to work, maybe that solution works for rest? 
smol.groupby('datetime_utc').sum().compute()